# <font color='green'> TUTORIAL </font>

## <font color='green'> 1. Installation in Unix </font> 

  - conda installation. Type in your console the following command:   
   ```bash
   wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
   ```
   
  - then add miniconda to your path   
   ```bash
   bash miniconda.sh -b -p $HOME/miniconda
   ```
   
  
  - create new virtual environment
   ```bash
   conda create -q -n qmworks python=3.5
   ```
    
  - Install dependecies
   ```bash 
   conda install --name qmworks -c anaconda hdf5
   conda install --name qmworks -c https://conda.anaconda.org/rdkit rdkit
   ```
    
  - Start environment
   ```bash
   source activate qmworks
   ``` 
   
  - install **qmworks** dependencies
    ```bash
     pip install https://github.com/SCM-NV/qmworks/tarball/master#egg=qmworks https://github.com/SCM-NV/plams/tarball/master#egg=plams --upgrade
    ```
### You are ready to start!

## <font color='green'> Starting the environment  </font>
Once *QMWORKS*  has been installed the user should run the following command to initialize the environment:

To leave the environment the following command is used

## <font color='green'> QMWorks Packages </font> 
Currently **QMWORKS** offers an interface with the following simulation software:
* #### SCM (ADF and DTFB)
* #### CP2K
* #### ORCA
* #### GAMESS-US
* #### DIRAC

If you are interested in having support for other packages, request it using the [github-issues](https://github.com/SCM-NV/qmworks/issues) system (Sorry but Gaussian is out of the menu).

The inteaction between aforementioned packages and ``qmworks`` is carry out through a set of python function: 
*adf, dftb, cp2k, orca, gamess and dirac*. These function invoked the quantum package using 2 mandatory arguments: a **Settings** object describing the input and a **Molecule** object containing the molecular geometry as will be detailed below.

Before we continue we should emphasize that it is the users resposability to install or load the simulation package that they want to use, in most supercomputer the simulation package are load using a command like:
```bash
load module superAwesomeQuantumPackage/3.1421
```
Also some simulation packages required that you configure a scratch folder. For instance *Orca* requires a ``SCR`` folder to be defnied while *ADF*  called it ``SCM_TMPDIR``.

## <font color='green'> QMWorks Settings </font> 
*Settings* are a subclass of python [dictionaries](https://docs.python.org/3.5/tutorial/datastructures.html#dictionaries) to represent herarchical structures, like 

<img src="files/simpleTree.png">

In [34]:
from qmworks import Settings

s = Settings()
s.b.z
s.c.f
s.c.g = 0

These hierachical resemble the input structure used in most quantum simulation package. For instance the basis set section in ADF is given by something like:
```
Basis
  Type SZ
  Core Large
End
```
We can resemble this structure using Setting,

In [35]:
s = Settings()
s.specific.adf.basis.basis = "DZP"
s.specific.adf.basis.core = "Large"

We are creating the *adf* herarchy under a key called *specific*, this key is used to differenciate keywords that are unique to a certain quantum package from those that can be used in several packages as we will see in the next section.

In the same way we can define *Settings* for all the sections
```
Basis
  Type SZ
End

Constraints
  Dist 1 2 1.0
End

Geometry
  Optim delocal
End

Integration
  Accint 6.0
End

Scf
  Converge 1e-06
  Iterations 100
End

Xc
  Lda
End
```
Represented by

In [47]:
# Basis
s.specific.adf.basis.basis = "DZP"
s.specific.adf.basis.core = "Large"

# Constrains
s.specific.adf.Constraints
print(s)

basis: 	DZP
specific: 	
         adf: 	
             Constraints: 	
             basis: 	
                   basis: 	DZP
                   core: 	Large



### <font color='green'> Generic Keywords </font> 
Many of the quantum chemistry packages use gaussian type orbitals (GTO) to perform the simulation (in contrast to the slater type orbitals). These package use the same standard for the basis set and it will be really handy if we can defined a "generic" keyword for basis sets.
Fortunately **qmworks** already offers such keyword that can be used among the packages that use the same basis standard

In [37]:
s = Settings()
s.basis = "DZP"

Internally **qmworks** will create a hierarchical structure representing the basis *DZP* for the packages that can handle that basis set.
Other generic keyowrds like: *functional*, *inithess*, etc. Have been implemented. 

### <font color='green'>  Templates </font> 

As has been shown so far, the ``Settings`` can be specified in two way: generic or specific. Generic keywords represent input properties that are present in most simulation packages like a *basis set* while *specific* keywords resemble the input structure of a given package.
 
*Generic* and *Specific* Settings are good but would it not be nice if we can predifined a set of templates for the most common quantum chemistry simulations like: single point calculations, geometry optimizations, transition state optimization, frequency calculations, etc.?
*qmworks* Also has a predefined set of templates that the user can use and modify. **Templates** are stored inside the **qmworks.templates** module and are load from *JSON* files. A JSON file is basically a nested dictionary that is translated to a **Settings** object by *qmworks*.

Below it is shown the defaults for single point calculation

In [50]:
single_point = {
    "specific": {
        "adf": {
            "basis": {"type": "SZ"},
            "xc": {"lda": ""},
            "integration": {"accint": 4.0},
            "scf": {
            "converge": 1e-6,
            "iterations": 100} },
        "dftb": {
            "task": {"runtype": "SP"},
            "dftb": {"resourcesdir": "DFTB.org/3ob-3-1"} },
        "cp2k" : {
          "force_eval": {
              "dft": {
                  "basis_set_file_name": "",
                  "mgrid": {
                      "cutoff": 400,
                      "ngrids": 4
                  },
                  "potential_file_name": "",
                  "print": {
                      "mo": {
                          "add_last"  : "numeric",
                          "each": {
                              "qs_scf": 0
                          },
                          "eigenvalues" : "",
                          "eigenvectors": "",
                          "filename": "./mo.data",
                          "ndigits": 36,
                          "occupation_numbers": ""
                      }
                  },
                  "qs": {
                      "method": "gpw"
                  },
                  "scf": {
                      "added_mos": "",
                      "eps_scf": 1e-06,
                      "max_scf": 200,
                      "scf_guess": "restart"
                  },
                  "xc": {
                      "xc_functional": "pbe"
                  }
              },
              "subsys": {
                  "cell": {
                      "periodic": "xyz"
                  },
                  "topology": {
                      "coordinate": "xyz",
                      "coord_file_name": ""
                  }
              }
          },
          "global": {
              "print_level": "low",
              "project": "qmworks-cp2k",
              "run_type": "energy_force"
          }
        },
        "dirac": {
            "DIRAC": "WAVEFUNCTION",
            "HAMILTONIAN": "LEVY-LEBLOND",
            "WAVE FUNCTION": "SCF"
        },
        "gamess": {
            "basis": {"gbasis": "sto", "ngauss": 3},
            "contrl": {"scftyp": "rhf", "dfttyp": "pbe"}
        },
        "orca": {
            "method": {
                "method": "dft",
                "functional": "lda"},
            "basis": {
                "basis": "sto_sz"}
        }
    }
}

If you are perfoming a bunch of *DFT* calculation for instance, you will only like to change the functional or basis set

In [39]:
from qmworks import templates
inp = templates.geometry.overlay(s)

The *overlay* method takes as input a template containing a default set for different packages and also takes the arguments provided by the user, as shown schematically 
<img src="files/merged.png">

This overlay method merged the defaults for a given packages (*ADF* in this case) with the input supplied by the user, always given preference to the user input
<img src="files/result_merged.png" width="700">

In [41]:
from plams import Molecule
acetonitrile = Molecule("files/acetonitrile.xyz")
print(acetonitrile)

  Atoms: 
    1         C      2.419290      0.606560      0.000000 
    2         C      1.671470      1.829570      0.000000 
    3         N      1.065290      2.809960      0.000000 
    4         H      2.000000      0.000000      1.000000 
    5         H      2.000000      0.000000     -1.000000 
    6         H      3.600000      0.800000      0.000000 



In [42]:
from qmworks import adf
job = adf(inp, acetonitrile)

### Extracting Properties
We can easily extract the *optimized geometry* from the *ADF* calculation using the following notation

In [43]:
optmized_mol_adf = job.molecule

We can use this geometry for further calculations using for instance another package

In [44]:
from qmworks import orca
s2 = Settings()
s2.specific.orca.main = "freq"
s2.specific.orca.basis.basis = 'sto_sz'
s2.specific.orca.method.functional = 'lda'
s2.specific.orca.method.method = 'dft'

job_freq = orca(s2, optmized_mol_adf)

frequencies = job_freq.frequencies

# <font color='blue'> Running the script </font>


The final python script looks like:

In [45]:
from qmworks import (adf, orca, run, Settings)
from plams import Molecule
import plams

def main():
    plams.init()
    acetonitrile = Molecule("files/acetonitrile.xyz")
    s = Settings()
    s.basis = "DZP"
    s.specific.adf.basis.core = "large"

    acetonitrile = Molecule("files/acetonitrile.xyz")
    job = adf(inp, acetonitrile)

    s2 = Settings()
    s2.specific.orca.main = "freq"
    s2.specific.orca.basis.basis = 'sto_sz'
    s2.specific.orca.method.functional = 'lda'
    s2.specific.orca.method.method = 'dft'

    job_freq = orca(s2, optmized_mol_adf)
    frequencies = job_freq.frequencies
    
    print(run(frequencies))

## A little discussion about graphs
*qmworks* is meant to be used for both workflow generation and execution,

<img src="files/simple_graph.png">

In [46]:
from qmworks import run


### <font color='green'> result = run(frequencies) </font>

Once you run the script, as you we will see in the next section, an input file for the *ADF* job is created

Running in **Cartesius** or **Bazis** through the *Slurm* resource manager can be done using and script like

The Slurm output looks like: